In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sn
import statsmodels.formula.api as smf
import pivottablejs as pivot

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)

## Excel Test - People Analytics Questions

1. Calculate average Stock Grant by job level, location and fiscal quarter							
2. Calculate average stock grant by job profile and location 							
3. Calculate the average stock grant by FBO/ GM (Functional Business Owner/ General Manager)and fiscal quarter		
4. Provide a summary of your findings. Feel free to include charts, tables or slides to best present your data. 	


I'm assuming Questions 1 to 3 are for pure reporting. There are different ways to provide a dynamic filter to flip these charts but for simplicity, I'm just using a bar and line charts. The recommendations should be at the bottom of this notebook.

In [2]:
df_input = pd.read_excel('./People Analytics Excel Test.xlsx',sheetname='Sheet1', skiprows = 9)
df_input = df_input.drop('Unnamed: 0', axis=1)
df_input['Quarter'] = pd.PeriodIndex(df_input['Hire Date'], freq='Q').strftime('Q%q')
print df_input.shape

(450, 9)


In [3]:
df_input.rename(columns={'Stock Granted as Amount ($USD)':'StockGrant',
                         'FBO/GM':'FBO', 'Job Profile ':'JobProfile','Job Level':'JobLevel'}, inplace=True)
df_input.head()

,Employee ID,Hire Date,JobProfile,JobLevel,Location,StockGrant,Currency,FBO,Quarter
0,1,2015-07-01,Customer Experience Program Manager 1,3,"Amsterdam, Netherlands",3500,USD,Head of Customers for Life,Q3
1,2,2015-07-01,Administration Senior Associate H,2,"San Francisco, United States",2500,USD,Head of People,Q3
2,3,2015-07-01,Experts Program Manager 2,4,Home Office: Germany,4750,USD,Head of Go to Market,Q3
3,4,2015-07-06,Senior Developer,3,"Sydney, Australia",2750,USD,Head of Tech Communication Tools,Q3
4,5,2015-07-06,Quality Engineer,2,"Sydney, Australia",2000,USD,Head of Engineering,Q3


In [4]:
s_column = 'StockGrant'

## Calculate Average Stock Grant by Job Level, Location and Fiscal quarter

In [5]:
#Average Stock Grant by Location / Quarter
pivot.pivot_ui(df_input, rows=['Location'], cols=['Quarter'], vals=['StockGrant'], 
               aggregatorName='Average', rendererName=['Line Chart'], outfile_path='pivottable1.html')

In [6]:
#Average Stock Grant by Job Level and Fiscal Quarter
pivot.pivot_ui(df_input, rows=['JobLevel'], cols=['Quarter'], vals=['StockGrant'], 
               aggregatorName='Average', rendererName=['Line Chart'], outfile_path='pivottable2.html')

## Average Stock Grant by Job Profile and Location

In [7]:
#Average Stock Grant by Job Profile and Fiscal Quarter
pivot.pivot_ui(df_input, rows=['Location','JobProfile'], cols=['Quarter'], vals=['StockGrant'], 
               aggregatorName='Average', rendererName=['Heatmap'], outfile_path='pivottable3.html')

## Average stock grant by FBO/ GM (Functional Business Owner/ General Manager)and fiscal quarter

In [8]:
#Average Stock Grant by FBO and Fiscal Quarter
pivot.pivot_ui(df_input, rows=['FBO'], cols=['Quarter'], vals=['StockGrant'], 
               aggregatorName='Average', rendererName=['Stacked Bar Chart'], outfile_path='pivottable4.html')

## Model

In [10]:
#Let us try to model the relations ship between Stock Grants, Location and the Job Level at those locations.
df_input.head()
df_input['JobLevel'] = df_input['JobLevel'].astype(str)

In [11]:
mod = smf.ols(formula='StockGrant ~ C(Location) + C(JobLevel)', data=df_input)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             StockGrant   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     339.3
Date:                Tue, 15 Aug 2017   Prob (F-statistic):          7.89e-224
Time:                        14:07:38   Log-Likelihood:                -3356.3
No. Observations:                 450   AIC:                             6743.
Df Residuals:                     435   BIC:                             6804.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------

In [12]:
#Most significant Attributes are
res.pvalues[res.pvalues < 0.05].sort_values(ascending=True)

C(JobLevel)[T.4]                                         2.141296e-187
C(JobLevel)[T.3]                                         3.606162e-125
C(JobLevel)[T.2]                                          4.212564e-49
C(Location)[T.San Francisco, United States]               1.044917e-26
C(Location)[T.Manila, Philippines]                        1.734145e-24
Intercept                                                 2.652157e-21
C(Location)[T.Home Office: Montana, United States]        4.373676e-03
C(Location)[T.Austin, United States]                      7.841907e-03
C(Location)[T.Home Office: California, United States]     3.970513e-02
dtype: float64

In [13]:
#Coefficient of those significant factors
res.params[res.pvalues < 0.05]

Intercept                                                 992.956998
C(Location)[T.Austin, United States]                      264.595482
C(Location)[T.Home Office: California, United States]     908.929741
C(Location)[T.Home Office: Montana, United States]       1246.838752
C(Location)[T.Manila, Philippines]                      -1504.804108
C(Location)[T.San Francisco, United States]              1150.734417
C(JobLevel)[T.2]                                         1010.204250
C(JobLevel)[T.3]                                         2025.221381
C(JobLevel)[T.4]                                         4348.113261
dtype: float64

## Recommendations / Insights:

We have a pretty robust model because the Adjusted R-Squared explains ~ 92% of the data we see. Some of the insights we can derive from this are as follows.

a)On an average across the company, if we disregard the influence of Job Level and Location, T1 level employees are being granted with $992 USD worth of stock options. Anything above that should be examined on why it is happening.

b)A T2 position hired in Manila, on an average, results in a net subtraction of 1010.20 - 1504.8 = -494.6 USD worth of stock options to the employees compensation package.

c) A T4 position hired in San Francisco on the other hand, on an average, results in a net addition of 4348.11 + 1150.73 = 5498.8 USD worth of stock options to the employees compensation package. Anything above that should be flagged for due diligence on the position and the value it brings to the company.

d) At Austin, we can see that positions T1, T2 and T4 result in an incremental 1010, 2025 and 4348 USD worth of stock options above the location average of 992.95 + 264.59 = 1257.5 USD worth of stock options.

e) Among the significant states in the US, Austin is definitely the cheapest in terms of additional Stock Grants.

f) We can go so far as to predict what kind of StockGrant offers would be a good one to make for each Location and Job Level. Actual stock grant offers exceed the predicted values can be used as a sort of threshold check for the business.